In [12]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.agents.tools import BaseTool
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from pydantic import BaseModel, Field
import json

class GreetingCheckPointInput(BaseModel):
    name: str = Field(..., description='username')

In [13]:

from typing import Optional, Type


llm = ChatOpenAI(model='gpt-3.5-turbo-0613')

def greeting_hello(name):
    return f'Hello {name}'

class GreetingHello(BaseTool):
    name = 'greeting_hello'
    description = "The name of the person to greet"

    def _run(self, name):
        result = greeting_hello(name)
        return result

    def _arun(self, name):
        raise NotImplementedError("This tool does not support async")

    args_schema: Optional[Type[BaseModel]] = GreetingCheckPointInput



In [3]:
!pip show langchain

Name: langchain
Version: 0.0.205
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /Users/takeuchitsubasa/.pyenv/versions/3.10.8/lib/python3.10/site-packages
Requires: aiohttp, async-timeout, dataclasses-json, langchainplus-sdk, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [14]:
from langchain.tools import format_tool_to_openai_function

tools = [GreetingHello()]
functions = [format_tool_to_openai_function(t) for t in tools]

In [15]:
functions[0]

{'name': 'greeting_hello',
 'description': 'The name of the person to greet',
 'parameters': {'title': 'GreetingCheckPointInput',
  'type': 'object',
  'properties': {'name': {'title': 'Name',
    'description': 'username',
    'type': 'string'}},
  'required': ['name']}}

In [17]:
from langchain.schema import HumanMessage, AIMessage

ai_message = llm.predict_messages([HumanMessage(content='挨拶をしてくれますか？')], functions=functions)

In [18]:
ai_message

AIMessage(content='', additional_kwargs={'function_call': {'name': 'greeting_hello', 'arguments': '{\n  "name": "ユーザー"\n}'}}, example=False)

In [8]:
ai_message.additional_kwargs['function_call']

{'name': 'greeting_hello', 'arguments': '{\n  "__arg1": "こんにちは"\n}'}

In [19]:
from langchain.schema import FunctionMessage

final_message = llm.predict_messages(
    [
        HumanMessage(content='挨拶をしてくれますか？'),
        ai_message,
        FunctionMessage(name='greeting_hello', content='Hello')
    ],
    functions=functions
)

In [20]:
final_message

AIMessage(content='こんにちは、ユーザーさん！どのようなお手伝いができますか？', additional_kwargs={}, example=False)

In [11]:
_args = json.loads(ai_message.additional_kwargs['function_call'].get('arguments'))
_args

{'__arg1': 'こんにちは'}

In [23]:
def get_current_weather(location, unit):
    weather_info = {
            "location": location,
            "temperature": "72",
            "unit": unit,
            "forecast": ["sunny", "windy"],
        }

    return json.dumps(weather_info)

class GetCurrentWeatherCheckInput(BaseModel):
    location: str = Field(..., description='city_and_state')
    unit: str = Field(..., description='sample')

class GetCurrentWeatherTool(BaseTool):
    name = 'get_current_weather'
    description = 'Specified location current weather acquisition.'

    def _run(self, location: str, unit: str):
        weather_info = {
            "location": location,
            "temperature": "72",
            "unit": unit,
            "forecast": ["sunny", "windy"],
        }

        return json.dumps(weather_info)

    def _arun(self, location: str, unit: str):
        raise NotImplementedError("This tool does not support async")

    args_schema: Optional[Type[BaseModel]] = GetCurrentWeatherCheckInput

In [24]:
tools = [GetCurrentWeatherTool()]
functions = [format_tool_to_openai_function(t) for t in tools]
functions[0]

{'name': 'get_current_weather',
 'description': 'Specified location current weather acquisition.',
 'parameters': {'title': 'GetCurrentWeatherCheckInput',
  'type': 'object',
  'properties': {'location': {'title': 'Location',
    'description': 'city_and_state',
    'type': 'string'},
   'unit': {'title': 'Unit', 'description': 'sample', 'type': 'string'}},
  'required': ['location', 'unit']}}